# Cyclistic Full Year Analysis
### author: Gordon Lam
### date: 6/3/2023

This analysis is the case study 1 of the Google Data Analytics Certificate's capstone project. 
In this case study, I am using the Divvy dataset and use them to help answer the business task "How do annual members and casual riders use Cyclistic bikes differently?"

### Business Objectives
- What is our goal?
    - to maxmize the number of annual riders
- What is our problem?
    - __how annual members and casual riders diff from each other?__
    - why would casual riders buy annual membership?
    - how can Cyclistic use digital media to influence casual riders to become annual members?
- Who is our main stakeholders
    - Lily Moreno (our director of marketing and manager)
    - Cyclistic Executive Team (responsible for deciding whether to approve the recommended marketing program)
    
### Business Task
The marketing team at Cyclistic wants to design a new marketing strategy to convert casual riders into annual members by understanding how annual members and casual riders use Cyclistic bikes differently


### Data
The Divvy dataset is located in amazonaws and they are organized by date. The data is reliable and organized as it comes from the company. And it is also comprehensive, current and cited. 

The dataset is secured as the data are tokenized so that the personal information of the users can be protected.

However, the majority in the column "start_station_id", "start_station_name", "end_station_id", "end_station_name" of June, August, and December is null. Regarding the data integrity of these data, they are removed in the combined dataset. However, with the latitude and longitude of the start and end stations, we can locate back the corresponding start and end stations name and id if in real life situations.

### Process
- created a column called "ride_length" by substracting the "ended_at" by "started_at" column
- created a column called "day_of_week" representing the day of the week given the date

In [1]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv
import glob
from pathlib import Path 

In [2]:
# paths
filepath = os.getcwd() + "/divvy-tripdata/"
csvpath = "csv/"
xlsxpath = "xlsx/"

In [3]:
def mergeCsv(filepath):
    df_list = []
    for file in glob.glob(filepath+csvpath+'*.csv'):
        df = pd.read_csv(file)
        df_list.append(df)
    combined_df = pd.concat(df_list)
    return combined_df

In [4]:
# merging all data
df = mergeCsv(filepath)
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,550CF7EFEAE0C618,electric_bike,2022-08-07 21:34:15,2022-08-07 21:41:46,NaN,NaN,NaN,NaN,41.930000,-87.690000,41.940000,-87.720000,casual
1,DAD198F405F9C5F5,electric_bike,2022-08-08 14:39:21,2022-08-08 14:53:23,NaN,NaN,NaN,NaN,41.890000,-87.640000,41.920000,-87.640000,casual
2,E6F2BC47B65CB7FD,electric_bike,2022-08-08 15:29:50,2022-08-08 15:40:34,NaN,NaN,NaN,NaN,41.970000,-87.690000,41.970000,-87.660000,casual
3,F597830181C2E13C,electric_bike,2022-08-08 02:43:50,2022-08-08 02:58:53,NaN,NaN,NaN,NaN,41.940000,-87.650000,41.970000,-87.690000,casual
4,0CE689BB4E313E8D,electric_bike,2022-08-07 20:24:06,2022-08-07 20:29:58,NaN,NaN,NaN,NaN,41.850000,-87.650000,41.840000,-87.660000,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190296,A303816F2E8A35A8,electric_bike,2023-01-11 17:46:23,2023-01-11 17:57:31,Clark St & Elm St,TA1307000039,Southport Ave & Clybourn Ave,TA1309000030,41.902634,-87.631591,41.920771,-87.663712,casual
190297,BCDBB142CC610382,classic_bike,2023-01-30 15:08:10,2023-01-30 15:33:26,Western Ave & Leland Ave,TA1307000140,Clarendon Ave & Gordon Ter,13379,41.966400,-87.688704,41.957867,-87.649505,member
190298,7D1C7CA80517183B,classic_bike,2023-01-06 19:34:50,2023-01-06 19:50:01,Clark St & Elm St,TA1307000039,Southport Ave & Clybourn Ave,TA1309000030,41.902973,-87.631280,41.920771,-87.663712,casual
190299,1A4EB636346DF527,classic_bike,2023-01-13 18:59:24,2023-01-13 19:14:44,Clark St & Elm St,TA1307000039,Southport Ave & Clybourn Ave,TA1309000030,41.902973,-87.631280,41.920771,-87.663712,casual


In [5]:
# Add day_of_week and ride_length column to the dataframe
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['day_of_week'] = df['started_at'].dt.dayofweek

ride_length = df['ended_at'] - df['started_at']
ride_length_formatted = ride_length.apply(lambda x: "{:02d}:{:02d}:{:02d}".format(int(x.total_seconds() // 3600), int((x.total_seconds() % 3600) // 60), int(x.total_seconds() % 60)))
df['ride_length'] = ride_length_formatted

In [6]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,day_of_week,ride_length
0,550CF7EFEAE0C618,electric_bike,2022-08-07 21:34:15,2022-08-07 21:41:46,NaN,NaN,NaN,NaN,41.93,-87.69,41.94,-87.72,casual,6,00:07:31
1,DAD198F405F9C5F5,electric_bike,2022-08-08 14:39:21,2022-08-08 14:53:23,NaN,NaN,NaN,NaN,41.89,-87.64,41.92,-87.64,casual,0,00:14:02
2,E6F2BC47B65CB7FD,electric_bike,2022-08-08 15:29:50,2022-08-08 15:40:34,NaN,NaN,NaN,NaN,41.97,-87.69,41.97,-87.66,casual,0,00:10:44
3,F597830181C2E13C,electric_bike,2022-08-08 02:43:50,2022-08-08 02:58:53,NaN,NaN,NaN,NaN,41.94,-87.65,41.97,-87.69,casual,0,00:15:03
4,0CE689BB4E313E8D,electric_bike,2022-08-07 20:24:06,2022-08-07 20:29:58,NaN,NaN,NaN,NaN,41.85,-87.65,41.84,-87.66,casual,6,00:05:52


## Understand the variables

In [7]:
# Understand the variables
variables = pd.DataFrame(columns=['Variable', 'Number of Unique values', 'values'])

for i, var in enumerate(df.columns):
    variables.loc[i] = [var, df[var].nunique(), df[var].unique().tolist()]
    
variables

,Variable,Number of Unique values,values
0,ride_id,6203371,"[550CF7EFEAE0C618, DAD198F405F9C5F5, E6F2BC47B..."
1,rideable_type,3,"[electric_bike, classic_bike, docked_bike]"
2,started_at,5240059,"[1659908055000000000, 1659969561000000000, 165..."
3,ended_at,5253907,"[1659908506000000000, 1659970403000000000, 165..."
4,start_station_name,1703,"[nan, DuSable Museum, Robert Fulton Elementary..."
5,start_station_id,1319,"[nan, KA1503000075, 819, 13084, 15623, 13256, ..."
6,end_station_name,1725,"[nan, Western Ave & Roscoe St, Western Ave & 1..."
7,end_station_id,1324,"[nan, 15634, 691, TA1309000067, 15623, 13084, ..."
8,start_lat,727582,"[41.93, 41.89, 41.97, 41.94, 41.85, 41.79, 41...."
9,start_lng,687194,"[-87.69, -87.64, -87.65, -87.72, -87.63, -87.6..."


### Data dictionary
|variable           |class  |description 
|:--------          |:----- |:-----------
|ride_id       |String |Id of the ride
|rideable_type |String |"eletric_bike", "classic_bike", "docked_bike"
|started_at        |Datetime |Start time of the ride in YYYY-MM-DD HH:MM:SS
|ended_at           |Datetime |End time of the ride in YYYY-MM-DD HH:MM:SS
|start_station_name |String |Name of starting station
|start_station_id   |String |Id of starting station
|end_station_name   |String |Name of ending station
|end_station_id     |String |Id of ending station
|start_lat |float | latitude of the starting station
|start_lng |float | longitude of the starting station
|end_lat |float | latitude of the ending station
|end_lng |float | longitude of the ending station
|member_casual |String | "member", "casual"
|ride_length |time |HH:MM:SS
|day_of_week |Integer |1-7, 1=Sunday, 7=Saturday

Some features will not be relevant to our analysis as there are too many missing values (start_station_name, start_station_id, end_station_name, end_station_id). Plus there are features that we do not need (ride_id)

In [8]:
# df.count() does not include NaN values
df2 = df[[column for column in df if df[column].count() / len(df) >= 0.3]]
del df2['ride_id']
print("List of dropped columns:", end=" ")
for c in df.columns:
    if c not in df2.columns:
        print(c, end=", ")
print('\n')

List of dropped columns: ride_id, 

